In [7]:
import sys
sys.path.append('../')
from lib.input_const import *
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import tikzplotlib

In [2]:
report = pd.read_csv(report_overview_file(202212), index_col=[0])

In [3]:
report

,industry,records-count-all,records-count-wo-outliers,entities-count,entities-with-later-debt,entities-with-later-severe-debt,sociodemographic-m-w-ratio,sociodemographic-unknown-gender-ratio,sociodemographic-age-mean,sociodemographic-age-stddev,...,amount-stddev,payment-days-diff-mean,payment-days-diff-stddev,payment-prepaid-days-mean,payment-prepaid-days-stddev,payment-prepaid-days-count,payment-delayed-days-mean,payment-delayed-days-stddev,payment-delayed-days-count,payment-on-time-count
Adbouti,COMPUTING,25739,25723,12832,1208,281,52.143670,14.116381,0.293536,1.680527,...,2419,4.354780,34.527702,30.642819,36.227364,11378,-17.379260,10.229324,13616,729
Addis,COLLECTION,69719246,69718491,2879096,277710,69664,51.130532,16.844818,0.007749,1.058606,...,0,-1.061049,5.721659,34.444642,53.593544,28153,-29.819211,2.170643,2513295,67177043
Adiningo,UNKNOWN,8507,8473,6910,433,112,55.841166,15.675461,0.117455,1.122557,...,48,-16.192848,21.372268,24.372376,42.002151,905,-21.231702,8.766257,7501,67
Alparal,BUILDING,22997,22997,10979,1280,274,55.635696,16.690248,0.277749,1.724208,...,1146,-12.813845,22.105732,21.377605,28.910181,4367,-21.174070,7.927887,18326,304
Apacun,UNKNOWN,4099428,4099428,885820,174756,44375,43.280281,14.628062,0.771140,0.346273,...,0,-30.000000,0.000000,NaN,NaN,0,-30.000000,0.000000,4099428,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Vertlei,BUILDING_SUPPLIER,136656,136656,4322,398,70,51.526927,13.857200,0.182355,15.804000,...,926,-12.466705,19.642994,18.053100,21.153394,22938,-18.871095,12.038245,112222,1496
Wamunit,CONSIGNMENT,87860,87860,62723,4321,494,39.694884,13.506721,0.791819,1.555748,...,0,-10.668336,3.603934,NaN,NaN,0,-10.670765,3.600746,87840,20
Waporing,UNKNOWN,5759,5759,4667,722,104,35.800879,10.918985,-0.659321,1.012981,...,0,-15.067720,10.574122,13.116618,15.243728,343,-17.031909,6.940441,5359,57
Wardmai,CONSIGNMENT,202010,201999,84045,13343,2075,35.638711,9.579877,0.275443,0.276266,...,1,3.379428,24.510504,23.083057,29.480872,78488,-10.021853,4.527026,112664,10847


In [4]:
COL_BAD_PAYERS_RATIO = 'bad-payers-percentage'
report[COL_BAD_PAYERS_RATIO] = 100.0 * report[OverviewReportColNames.EntitiesWithLaterDebt] / report[OverviewReportColNames.EntitiesCount]

In [10]:
sb.scatterplot(
    report.loc[report[OverviewReportColNames.RecordsCountAll] > 1000], 
    x=OverviewReportColNames.PaymentDaysMean, 
    y=COL_BAD_PAYERS_RATIO, size=OverviewReportColNames.PaymentDaysStddev)
# plt.show()

ValueError: Could not interpret value `payment-days-diff-stddevpayment-days-diff-stddevpayment-days-diff-stddevpayment-days-diff-stddevpayment-days-diff-stddevpayment-days-diff-stddevpayment-days-diff-stddevpayment-days-diff-stddevpayment-days-diff-stddevpayment-days-diff-stddev` for parameter `size`

In [9]:
tikzplotlib.save("test.tex")

<Figure size 640x480 with 0 Axes>

In [12]:
report.loc[report[OverviewReportColNames.RecordsCountAll] > 1000].sort_values([COL_BAD_PAYERS_RATIO], ascending=False)

,industry,records-count-all,records-count-wo-outliers,entities-count,entities-with-later-severe-debt,sociodemographic-m-w-ratio,sociodemographic-unknown-gender-ratio,sociodemographic-age-mean,sociodemographic-age-stddev,sociodemographic-age-skewness,...,payment-days-diff-mean,payment-days-diff-stddev,payment-prepaid-days-mean,payment-prepaid-days-stddev,payment-prepaid-days-count,payment-delayed-days-mean,payment-delayed-days-stddev,payment-delayed-days-count,payment-on-time-count,bad-payers-percentage
Numtleal,GASTRO,3379,3379,506,88,45.785479,11.946395,0.735408,3.857143,0.260870,...,3.152116,22.897405,24.481629,23.147336,1252,-14.255167,8.393505,1403,724,17.391304
Nyurever,CONSIGNMENT,1566,1566,1466,132,39.818565,13.677756,-0.451933,2.101695,0.001364,...,17.495530,37.234145,38.391167,33.427330,951,-17.727626,6.916099,514,101,9.004093
Bernev,CONSIGNMENT,1312964,1312964,281239,21284,51.317688,15.671795,0.275507,0.515905,0.003684,...,0.000000,0.000000,NaN,NaN,0,NaN,NaN,0,1312964,7.567940
Otheso,UNKNOWN,6501804,6501804,847770,57794,43.372366,13.983529,0.169654,0.258703,0.000110,...,0.000000,0.000000,NaN,NaN,0,NaN,NaN,0,6501804,6.817179
Apacun,UNKNOWN,4099428,4099428,885820,44375,43.280281,14.628062,0.771140,0.346273,0.006999,...,-30.000000,0.000000,NaN,NaN,0,-30.000000,0.000000,4099428,0,5.009483
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Fiverfer,UNKNOWN,1514,1514,1225,2,50.612448,12.231176,0.131831,0.802360,0.002449,...,-0.984808,5.406993,3.855046,4.260669,545,-4.322503,3.833410,831,138,0.163265
Ensenmu,UNKNOWN,25023,25015,7434,12,38.549072,6.573745,0.387766,0.046433,0.005650,...,-5.232780,12.806988,5.703465,12.528537,7591,-10.863299,9.532693,16035,1389,0.161421
Muexnau,UNKNOWN,18181,18180,13974,10,49.822837,13.701658,-0.231882,0.361550,0.002075,...,-3.732563,7.224004,4.304690,11.176238,3561,-6.168854,3.633941,13485,1134,0.071561
Tencedeu,UNKNOWN,9172,9172,4703,3,45.757240,12.108848,-0.350385,0.031470,0.003402,...,-2.226341,8.524676,4.843861,8.434637,3087,-6.542075,5.904896,5407,678,0.063789


In [12]:
report.sort_values(
    OverviewReportColNames.RecordsCountAll, ascending=False).head(10)[
    [OverviewReportColNames.RecordsCountWithoutOutliers, 
     OverviewReportColNames.EntitiesWithLaterDebt, 
     OverviewReportColNames.GendersRatio, 
     OverviewReportColNames.AgeMean, 
     OverviewReportColNames.PaymentDaysMean, 
     OverviewReportColNames.AmountMean]
]

,records-count-wo-outliers,entities-with-later-debt,sociodemographic-m-w-ratio,sociodemographic-age-mean,payment-days-diff-mean,amount-mean
Addis,69718491,277710,51.130532,0.007749,-1.061049,0
Estry,36659140,206318,40.160288,0.920158,-1.122091,0
Ganpe,14364531,120314,56.645426,-0.154343,0.000000,0
Tionse,8359220,96466,43.733661,0.501676,-0.223942,0
Parfae,8135856,83171,40.781915,1.495100,-7.513751,0
Otheso,6501804,182033,43.372366,0.169654,0.000000,0
Ningui,4922236,138259,39.571976,0.584500,-30.000000,0
Apacun,4099428,174756,43.280281,0.771140,-30.000000,0
Everna,3884437,214347,41.602085,0.559599,6.015259,0
Picfio,3311629,19729,53.228795,0.193320,-2.758926,0
